In [ ]:
#run this cell to install qiskit 
#skip this cell if you have pre-installed qiskit library or if you are running on IBM-Quantum
! pip install qiskit

# <center>Quantum teleportation protocol</center>
Welcome to this lab on quantum teleportation in which we will solve a problem that Alice and Bob have been having. Alice possesses a qubit in an unknown state  |𝜓⟩
  and she wishes to transfer this quantum state to Bob. However, they are very far apart and lack any means to transfer quantum information directly, only classical information. Is it possible to achieve their goal?

It turns out that if Alice and Bob share an entangled qubit pair, she can transfer her qubit state to Bob by sending two bits of classical information. This process is known as teleportation because, at the end, Bob will possess |ψ⟩, and Alice will no longer have it.

In [ ]:
#The following statements import useful libraries containing functions and methods that will help us build the circuit.
from qiskit.circuit import QuantumCircuit,QuantumRegister,ClassicalRegister
from qiskit import execute,transpile
from qiskit.extensions import Initialize
from qiskit_aer import Aer
from qiskit.visualization import *
from qiskit.quantum_info import *
import numpy as np
import matplotlib.pyplot as plt
from qiskit.tools.jupyter import *
print('Libraries imported successfully')

##  Implementation 

--- 

In this protocol a 3-qubit quantum circuit is created where Alice holds two qubits and Bob holds one qubit.<br>
qubit which Alice wants to teleport is represented as psi, Entangled qubit that Alice has is represented as 'a' and Bob's qubit is represented as 'b'.


#### *Step 1 :*
- Alice and Bob shares an entangled pair of qubits known as Bell states.
- So Alice and Bob share each qubit from an entangled Bell pair.
- Bell pair can be creates by the following steps:
    - *transfer one of the qubit to X-basis* $(\ket{+} or \ket{-})$ *using Hadamard gate.*
    - *apply CNOT gate onto the other qubit controlled by the one in the X-basis.*
 

In [ ]:
def create_bell_pair(qc,a,b):
    '''This function creates a bell pair in quantum circuit qc using qubits a and b'''
    qc.h(a)
    qc.cx(a,b)
    

In [ ]:
psi=QuantumRegister(1,'𝜓')       #teleporting qubit
a=QuantumRegister(1,'alice')     #alice qubit
b=QuantumRegister(1,'bob')       #bob qubit
crz,crx=ClassicalRegister(1,'crz'),ClassicalRegister(1,'crx')
teleportation_circuit=QuantumCircuit(psi,a,b,crx,crz)

# step 1
# Now we create bell states

create_bell_pair(teleportation_circuit,a,b)

# draw the circuit
teleportation_circuit.draw(output='mpl')


#### *Step 2 :*
Now Alice creates a Bell basis on her two qubits as follows :
- *Alice applies a CNOT gate to `alice` qubit, controlled by* $\ket{\psi}$ *(the qubit she is trying to send Bob).*
- *Then Alice adds a Hadamard gate to* $\ket{\psi}$*(psi)*.

In [ ]:
def alice_gates(qc,psi,a):
    ''' This function takes quantum circuit, psi and a qubits as argument and add alice gates(bell basis)'''
    qc.cx(psi,a)
    qc.h(psi)

In [ ]:
psi=QuantumRegister(1,'𝜓')       #teleporting qubit
a=QuantumRegister(1,'alice')     #alice qubit
b=QuantumRegister(1,'bob')       #bob qubit
crz,crx=ClassicalRegister(1,'crz'),ClassicalRegister(1,'crx')
teleportation_circuit=QuantumCircuit(psi,a,b,crx,crz)

# step 1
# create bell states 

create_bell_pair(teleportation_circuit,a,b)

# step 2
# add alice gates

alice_gates(teleportation_circuit,psi,a)

# draw the circuit
teleportation_circuit.draw(output='mpl')

#### *Step 3 :*
- *Next, Alice applies a measurement to both qubits that she owns, `alice`
 and `psi`, and stores this result in two classical bits.*

 <br>She then sends these two bits to Bob.

In [ ]:
def measurement(qc,psi,a):
    '''This function takes quantum circuit, psi and a qubits as arguments and measure qubits that Alice owns'''
    qc.measure(psi,0)
    qc.measure(a,1)

In [ ]:
psi=QuantumRegister(1,'𝜓')       #teleporting qubit
a=QuantumRegister(1,'alice')     #alice qubit
b=QuantumRegister(1,'bob')       #bob qubit
crz,crx=ClassicalRegister(1,'crz'),ClassicalRegister(1,'crx')
teleportation_circuit=QuantumCircuit(psi,a,b,crx,crz)

# step1 
# create bell states  

create_bell_pair(teleportation_circuit,a,b)

# step 2
# add alice gates

alice_gates(teleportation_circuit,psi,a)

# step 3
# measure and send alice qubits

measurement(teleportation_circuit,psi,a)

# draw the circuit
teleportation_circuit.draw(output='mpl')

---

#### *Step 4 :*
In this step, Bob, who is already in possession of bob qubit, dynamically adds specific gates to the circuit based on the state of the classical bits received from Alice:

- *If the bits are `00`, no action is required.*
- *If they are `01`, an 𝑋 gate (also known as a Pauli-X or a bit-flip gate) should be applied.*
- *For bits `10`, a 𝑍 gate (also known as a Pauli-Z or a phase-flip gate) should be applied.*
- *Lastly, if the classical bits are `11`, a combined 𝑍𝑋 gate should be applied, which involves applying both the 𝑍 and 𝑋 gates in sequence.*

---

To add these gates controlled by classical bits we can use two methods :
- *First one is `c_if()`.*
- *Second one is `if_test(())`*


Here we use `c_if()`.
Its syntax is as follows:<br>
`circuit.gate().c_if(classical bit index, classical bit value)`
- *If the classical bit we mentioned in c_if is equal to the mentioned value(0 or 1), then the gate we mentioned will be added.It is similar to if statement in programming.*

---

In [ ]:
def bob_gates(qc,b,crx,crz):
    # 
    # Here we use c_if to control our gates with a classical
    # bit instead of a qubit
    qc.z(b).c_if(crz,1)
    qc.x(b).c_if(crx,1)
    

In [ ]:
psi=QuantumRegister(1,'𝜓')       #teleporting qubit
a=QuantumRegister(1,'alice')     #alice qubit
b=QuantumRegister(1,'bob')       #bob qubit
crz,crx=ClassicalRegister(1,'crz'),ClassicalRegister(1,'crx')
teleportation_circuit=QuantumCircuit(psi,a,b,crx,crz)

# step 1
# create bell states 

create_bell_pair(teleportation_circuit,a,b)
teleportation_circuit.barrier() # barrier to separate steps for easier understanding

# step 2
# add alice gates

alice_gates(teleportation_circuit,psi,a)
teleportation_circuit.barrier() # barrier to separate steps for easier understanding

# step 3
# measure and send alice qubits

measurement(teleportation_circuit,psi,a)
teleportation_circuit.barrier() # barrier to separate steps for easier understanding

# step 4
# add bob gates

bob_gates(teleportation_circuit,b,crx,crz)

# draw the circuit
teleportation_circuit.draw(output='mpl')


---

## Simulating and verifying the protocol

Whether our circuit is teleporting the desired quantum state or not, we will now simulate and verify the protocol.

- *First, we generate a random state using `random_statevector()`.*
- *Secondly, we initialize `psi` qubit with the generated state and perform teleportation protocol.*
- *Then, we verify the state of Bob qubit and plot bloch sphere.*

---

#### *Step 1 :*
Generate a random state using `random_statevector()`.

In [ ]:
psi_state=random_statevector(2)

# view the generated statevector

display(array_to_latex(psi_state,prefix="|\\psi\\rangle ="))

# plot a bloch shphere
plot_bloch_multivector(psi_state)

#### *step 2 :*
Initialize qubit `psi` with the generated random state and perform teleportation protocol.

In [ ]:
# ---------------------------------------------------------------------------------------------------------------------------------------------
# step 1

# Create random 1-qubit state
psi_state=random_statevector(2)

#display the statevector
display(array_to_latex(psi_state))

#plot the bloch sphere
plot_bloch_multivector(psi_state)

In [ ]:

#--------------------------------------------------------------------------------------------------------------------
## STEP 2

psi=QuantumRegister(1,'𝜓')       #teleporting qubit
a=QuantumRegister(1,'alice')     #alice qubit
b=QuantumRegister(1,'bob')       #bob qubit
crz,crx=ClassicalRegister(1,'crz'),ClassicalRegister(1,'crx')
teleportation_circuit=QuantumCircuit(psi,a,b,crx,crz)

# initializing state to qubit psi

teleportation_circuit.initialize(psi_state,psi)
teleportation_circuit.barrier() # barrier to separate steps for easier understanding



# step 1
# create bell states 

create_bell_pair(teleportation_circuit,a,b)
teleportation_circuit.barrier() # barrier to separate steps for easier understanding

# step 2
# add alice gates

alice_gates(teleportation_circuit,psi,a)
teleportation_circuit.barrier() # barrier to separate steps for easier understanding

# step 3
# measure and send alice qubits

measurement(teleportation_circuit,psi,a)


# step 4
# add bob gates

bob_gates(teleportation_circuit,b,crx,crz)

# draw the circuit
teleportation_circuit.draw(output='mpl')

#----------------------------------------------------------------------------------------------------------------------


#### *step 3 :*
*Save the statevector, measure it and verify the teleported state.*

In [ ]:
teleportation_circuit.save_statevector()

sim=Aer.get_backend('aer_simulator')

final_state=sim.run(teleportation_circuit).result().get_statevector()

plot_bloch_multivector(final_state)

#### *The random state we generated is :*

In [ ]:
plot_bloch_multivector(psi_state)

#### *As you can see here, the state of qubit psi is successfully teleported to Bob's qubit.*<br>
*Please re-run the above cells in case the same state is not teleported.*

---

## <font color=orange> Now it's your turn </font>

#### First prepare a teleportaion circuit.

#### *step 1 :*
- *create a function `bell_states()` which prepares Bell states by taking quantum circuit, Alice qubit and Bob qubit as parameters.*

---



In [ ]:
# -----------------------------------------------



# 
# your code goes here
# 





# -----------------------------------------------

---

#### *step 2 :*<i>
- create a function `alice_gates()` which adds following gates by taking quantum circuit, psi and Alice qubit as parameter.
    - *apply a CNOT gate to Alice qubit, controlled by* $\ket{\psi}$.
    - *Then add a Hadamard gate to* $\ket{\psi}$*(psi)*.</i>

---

In [ ]:
# -----------------------------------------------------------------------




# 
# your code goes here
# 







# -----------------------------------------------------------------------

---

#### *step 3 :*<i>
- now create a function `measure_alice_qubits()` which performs following measurement by taking quantum circuit,Alice qubit, psi and classical bits crz and crx as input parameters:
    - measure psi and store the result into crx bit.
    - measure Alice qubit and store the result into crz bit.</i>

---

In [ ]:
# ------------------------------------------------------------------------




# 
# your code goes here
# 








# -------------------------------------------------------------------------

---

#### *step 4 :*
- *create a function `bob_gates()` which performs following operations by taking quantum circuit, Bob qubit, crz bit and crx bit as input parameters.*
    - *If the bits are `00`, no action is required.*
    - *If they are `01`, an 𝑋 gate (also known as a Pauli-X or a bit-flip gate) should be applied.*
    - *For bits `10`, a 𝑍 gate (also known as a Pauli-Z or a phase-flip gate) should be applied.*
    - *Lastly, if the classical bits are `11`, a combined 𝑍𝑋 gate should be applied, which involves applying both the 𝑍 and 𝑋 gates in sequence.*

---


In [ ]:
# -------------------------------------------------------------------------





# 
# your code goes here
# 







# ---------------------------------------------------------------------------

---

#### *Now create a quantum circuit named `qc` with 3 qubits and classical bits where q0 represents qubit psi, q1 represents Alice qubit and q2 represents bob qubit, c0 represents `crx` bit and c1 represents `crz` bit.*<br><br>

#### *Then add the following functions in order and draw the circuit.*
1) `bell_states()`
2) `alice_gates()`
3) `measure_alice_qubits()`
4) `bob_gates()`
5) draw the circuit

---

In [ ]:
# ---------------------------------------------------------------------------








# 
# your code goes here
# 













# -----------------------------------------------------------------------------

#### *your circuit should be similar to the below circuit.*

![teleportation circuit](beta\Tutorials\circuit_images\teleportation_circuit.png)

---

#### Now simulate the quantum teleportation 
*step 1:*
- *generate a single qubit random statevector using `random_statevector()` function and store in a variable.*
- *plot a bloch sphere using `plot_bloch_multivector()` for the generated statevector.*

---


In [ ]:
# -------------------------------------------------------------------------





# 
# your code goes here
# 







# ---------------------------------------------------------------------------

---

#### *step 2 :*
- *create the previous teleportation circuit by initializing the qubit 0 (psi) with the generated random statevector immediately after creating quantum circuit.* 

---

In [ ]:
# -------------------------------------------------------------------------





# 
# your code goes here
# 







# ---------------------------------------------------------------------------

---

#### *step 3 :*
*Save the statevector, measure it and plot the bloch sphere.*

----

In [ ]:
# -------------------------------------------------------------------------





# 
# your code goes here
# 







# ---------------------------------------------------------------------------

#### *You should be seeing the same qubit state teleported from qubit 0 to qubit 2. If not, please re-run the above cells or check the circuit again.*

---

*Content written by- Akash*